# Parkeervakken in Amsterdam

In [1]:
import sys, os
print(f'python     {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}')

import pandas as pd
print(f'pandas     {pd.__version__}')

import geopandas as gpd
print(f'geopandas  {gpd.__version__}')

from shapely.geometry import Point, Polygon, MultiPolygon
print(f'shapely    {sys.modules["shapely"].__version__}')

import matplotlib.pyplot as plt
print(f'matplotlib {sys.modules["matplotlib"].__version__}')

import folium
print(f'folium     {folium.__version__}')

python     3.6.6
pandas     0.23.4
geopandas  0.4.0
shapely    1.6.4.post2
matplotlib 2.2.2
folium     0.7.0


## Parkeervakken

In [2]:
# lees data
parkeervakken = gpd.read_file('../shapefiles/alle_parkeervakken.shp')
parkeervakken.drop(['id', 'geo_id', 'straatnaam', 'soort', 'type', 'e_type', 'bord'], axis=1, inplace=True)
parkeervakken.dropna(inplace=True)

In [3]:
# sample data
parkeervakken.sample(n=3)

,buurtcode,stadsdeel,aantal,geometry
80583,E21a,west,1,"POLYGON ((118924.85 486039.59, 118924.03 48603..."
6507,F84a,nieuw_west_nf,1,"POLYGON ((114015.42 485125.14, 114016.99 48512..."
21740,F80a,nieuw_west_nf,1,"POLYGON ((115091.98 486931.7, 115093.57 486926..."


In [4]:
# aantal parkeervakken per buurt
buurt_aantal = parkeervakken[['buurtcode', 'aantal']].groupby(by='buurtcode').sum()

In [5]:
# sample data
buurt_aantal.sample(n=3)

,aantal
buurtcode,
K23a,198
M57c,427
E13d,268


## Buurten

In [6]:
# lees data
buurt = gpd.read_file('../shapefiles/brtk2010_ind2005_region.shp')
buurt.drop(['BUURT', 'BC', 'LOKATIE', 'OPP', 'SD09', 'REGIEBUURT', 'REGIEBUU0', 'VEILIGHEID'], axis=1, inplace=True)
buurt.columns=['buurt', 'buurtcode', 'geometry']

In [7]:
# aantal hectare per buurt
buurt['ha'] = buurt.geometry.area / 100**2

In [8]:
# sample data
buurt.sample(n=3)

,buurt,buurtcode,geometry,ha
97,Marnixbuurt Zuid,A06j,"POLYGON ((120232.0450000036 486855.7200000035,...",5.108284
197,Overtoomse Veld Noord,F86a,"POLYGON ((117543.5900000036 487007.0250000035,...",34.589878
306,Reigersbos Midden,T96f,"POLYGON ((126269.2850000036 478167.5900000035,...",25.097425


## Merge

In [9]:
# merge
geodata = pd.merge(buurt, buurt_aantal, how='left', on='buurtcode').fillna(0)
geodata.head()

,buurt,buurtcode,geometry,ha,aantal
0,Van der Pekbuurt,N60a,"POLYGON ((121744.7100000036 488409.8000000035,...",57.971628,1756.0
1,Bloemenbuurt Zuid,N60b,"POLYGON ((122466.2000000036 490326.9900000035,...",36.658592,1030.0
2,Bloemenbuurt Noord,N60c,"POLYGON ((122466.2000000036 490326.9900000035,...",36.788222,1030.0
3,IJplein e.o.,N61a,"POLYGON ((122072.4000000036 488339.4000000035,...",41.221527,1219.0
4,Vogelbuurt Zuid,N61b,"POLYGON ((122530.9900000036 488489.0000000036,...",33.966420,1136.0


In [10]:
# sample data
geodata.sample(n=3)

,buurt,buurtcode,geometry,ha,aantal
314,Vogeltjeswei,T93j,"POLYGON ((125987.3000000036 481273.3850000036,...",7.363779,255.0
225,Diamantbuurt,K26a,"POLYGON ((122010.6900000036 485028.0150000035,...",17.041204,772.0
306,Reigersbos Midden,T96f,"POLYGON ((126269.2850000036 478167.5900000035,...",25.097425,703.0


## Aantal parkeerplaatsen per buurt

In [11]:
m = folium.Map(location = [52.3714942, 4.895862], zoom_start = 12)

folium.Choropleth(
    geo_data = geodata,
    name = 'geometry',
    data = geodata,
    columns = ['buurtcode', 'aantal'],
    key_on = 'feature.properties.buurtcode',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'aantal_per_buurt.html'))

m

## Aantal parkeerplaatsen per KM$^2$

In [12]:
geodata['aantal_per_ha'] = geodata.aantal / geodata.ha

In [13]:
geodata.sample(n=3)

,buurt,buurtcode,geometry,ha,aantal,aantal_per_ha
445,Leidsebuurt Zuidoost,A07d,"POLYGON ((120748.2750000036 486222.7250000035,...",5.223733,52.0,9.954567
177,Filips van Almondekwartier,E40e,"POLYGON ((119016.9850000036 487192.9800000035,...",4.335449,0.0,0.000000
298,G-buurt Oost,T94j,"POLYGON ((127180.3050000036 481903.5200000035,...",97.227200,525.0,5.399724


In [14]:
m = folium.Map(location = [52.3714942, 4.895862], zoom_start = 12)

folium.Choropleth(
    geo_data = geodata,
    name = 'geometry',
    data = geodata,
    columns = ['buurtcode', 'aantal_per_ha'],
    key_on = 'feature.properties.buurtcode',
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.8,
    legend_name = 'Aantal parkeervakken per hectare',
    reset = True).add_to(m)

m.save(os.path.join('..', 'output', 'aantal_per_hectare.html'))

m